In [11]:
from datasets import load_dataset
import numpy as np
import os
import glob
from PIL import Image

## Convert images to indices and set resolution

In [60]:
raw_images_path = "../datasets/raw_media/gameplay_images"
resized_images_path = "../datasets/resized_media/gameplay_images"

 # Define the scale factor
scale_factor = 0.5

# Iterate over each folder in the images path
for folder in os.listdir(raw_raw_images_path):
    # Check if the current item is a directory
    if os.path.isdir(os.path.join(raw_images_path, folder)):
       
        # Get all jpg files in the current folder
        image_files = glob.glob(os.path.join(raw_images_path, folder, '*.jpg')) + glob.glob(os.path.join(raw_images_path, folder, '*.png'))
        
        # Iterate over each file in the current folder
        for i, file in enumerate(image_files):
            
            # Scale the resolution of the image
            img = Image.open(file)
            img = img.resize((int(img.size[0]*scale_factor), int(img.size[1]*scale_factor)))
            
            # Save the image with the same extension as the original file
            file_extension = os.path.splitext(file)[1]
            save_folder = os.path.join(resized_images_path, folder)
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
            img.save(os.path.join(save_folder, f'{i}{file_extension}'))
            
            # # Print the image resolution
            # print(f'Image resolution: {img.size[0]} x {img.size[1]}')